# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv("cities.csv")
cities_df.dropna()
cities_df.head()


,Unnamed: 0,city,lat,lon,temp,humidity,wind,cloud,country,date
0,0,bluff,-46.6000,168.3333,54.00,91,10.00,86,NZ,1611012807
1,1,iqaluit,63.7506,-68.5145,21.20,79,16.11,90,CA,1611012561
2,2,rikitea,-23.1203,-134.9692,78.78,73,13.02,43,PF,1611012807
3,3,valdivia,-39.8142,-73.2459,80.60,47,10.36,0,CL,1611012808
4,4,victoria,22.2855,114.1577,57.00,63,17.09,47,HK,1611012809


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations
locations = cities_df[["lat", "lon"]]
humidity = cities_df["humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
weather_file=cities_df.loc[(cities_df['temp']>=70)&(cities_df['temp']<=80)]
wind_file=weather_file.loc[(weather_file['wind']<=10)]
ideal_weather=wind_file.loc[(wind_file['cloud']==0)]
ideal_weather.dropna
ideal_weather.head()

,Unnamed: 0,city,lat,lon,temp,humidity,wind,cloud,country,date
23,23,caravelas,-17.7125,-39.2481,76.32,83,7.34,0,BR,1611012815
31,31,pisco,-13.7000,-76.2167,73.40,73,8.05,0,PE,1611012818
82,82,el burrion,25.5333,-108.4000,79.00,32,3.00,0,MX,1611012835
86,86,yenagoa,4.9247,6.2642,77.41,89,4.47,0,NG,1611012836
96,96,saint-philippe,-21.3585,55.7679,78.80,73,5.75,0,RE,1611012838


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df


,Unnamed: 0,city,lat,lon,temp,humidity,wind,cloud,country,date,Hotel Name
0,23,caravelas,-17.7125,-39.2481,76.32,83,7.34,0,BR,1611012815,
1,31,pisco,-13.7000,-76.2167,73.40,73,8.05,0,PE,1611012818,
2,82,el burrion,25.5333,-108.4000,79.00,32,3.00,0,MX,1611012835,
3,86,yenagoa,4.9247,6.2642,77.41,89,4.47,0,NG,1611012836,
4,96,saint-philippe,-21.3585,55.7679,78.80,73,5.75,0,RE,1611012838,
5,139,saint-joseph,-21.3667,55.6167,78.80,73,5.75,0,RE,1611012856,
6,187,tamale,9.4008,-0.8393,79.61,16,5.55,0,GH,1611012684,
7,306,saint-leu,-21.1500,55.2833,78.80,73,5.75,0,RE,1611012917,
8,329,warri,5.5167,5.7500,77.81,87,2.21,0,NG,1611012923,
9,367,cradock,-32.1642,25.6192,70.45,63,2.15,0,ZA,1611012934,


In [8]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["lat"]
    lng = row["lon"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [22]:
hotel_df

,Unnamed: 0,city,lat,lon,temp,humidity,wind,cloud,country,date,Hotel Name
0,23,caravelas,-17.7125,-39.2481,76.32,83,7.34,0,BR,1611012815,Pousada Casa de Maria
1,31,pisco,-13.7000,-76.2167,73.40,73,8.05,0,PE,1611012818,"Hotel Paracas, a Luxury Collection Resort, Par..."
2,82,el burrion,25.5333,-108.4000,79.00,32,3.00,0,MX,1611012835,El Sembrador Hotel
3,86,yenagoa,4.9247,6.2642,77.41,89,4.47,0,NG,1611012836,"Ebiis Hotel Etegwe, Yenagoa"
4,96,saint-philippe,-21.3585,55.7679,78.80,73,5.75,0,RE,1611012838,Palm Hotel & Spa
5,139,saint-joseph,-21.3667,55.6167,78.80,73,5.75,0,RE,1611012856,Palm Hotel & Spa
6,187,tamale,9.4008,-0.8393,79.61,16,5.55,0,GH,1611012684,Jeyads Lodge
7,306,saint-leu,-21.1500,55.2833,78.80,73,5.75,0,RE,1611012917,Palm Hotel & Spa
8,329,warri,5.5167,5.7500,77.81,87,2.21,0,NG,1611012923,BON Hotel Delta
9,367,cradock,-32.1642,25.6192,70.45,63,2.15,0,ZA,1611012934,Die Tuishuise and Victoria Manor


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))